In [1]:
import numpy as np
from pyomo.environ import *
import pandas as pd

In [2]:
model = ConcreteModel()

In [3]:
model.plants = RangeSet(1, 3)
model.dcs = RangeSet(1, 12)

capacity = {1: 120, 2: 90, 3: 100}
model.capacity = Param(model.plants, initialize=capacity)

prod_cost = {1: 8, 2: 7, 3: 9}
model.prod_cost = Param(model.plants, initialize=prod_cost)

In [4]:
trans_cost = {
    (1,1):2, (1,2):3, (1,3):2, (1,4):3, (1,5):5, (1,6):4.4, (1,7):3.2, (1,8):3, (1,9):1.5, (1,10):3, (1,11):6.0, (1,12):1.0,
    (2,1):1.9, (2,2):3, (2,3):2.4, (2,4):2.4, (2,5):2, (2,6):4.1, (2,7):2.8, (2,8):3, (2,9):1.7, (2,10):2, (2,11):1.0, (2,12):4.5,
    (3,1):3.1, (3,2):1.8, (3,3):2.8, (3,4):3.4, (3,5):2, (3,6):2.9, (3,7):2.2, (3,8):3.2, (3,9):2, (3,10):2.8, (3,11):4.5, (3,12):6.0
}

In [5]:
model.trans_cost = Param(model.plants, model.dcs, initialize=trans_cost)

demand = {1:20, 2:30, 3:40, 4:35, 5:15, 6:19, 7:25, 8:20, 9:18, 10:12, 11:30, 12:40}
model.demand = Param(model.dcs, initialize=demand)

model.selling_price = Param(initialize=20)

model.x = Var(model.plants, model.dcs, domain=NonNegativeReals)

In [6]:
def obj_rule(model):
    return sum((model.selling_price - model.prod_cost[i] - model.trans_cost[i,j]) * model.x[i,j] 
               for i in model.plants for j in model.dcs)
model.objective = Objective(rule=obj_rule, sense=maximize)

In [7]:
def capacity_rule(model, i):
    return sum(model.x[i,j] for j in model.dcs) <= model.capacity[i]
model.capacity_constraint = Constraint(model.plants, rule=capacity_rule)

In [8]:
def demand_rule(model, j):
    return sum(model.x[i,j] for i in model.plants) == model.demand[j]

In [9]:
model.demand_constraint = Constraint(model.dcs, rule=demand_rule)

solver = SolverFactory('glpk')
results = solver.solve(model)

if results.solver.status == SolverStatus.ok and results.solver.termination_condition == TerminationCondition.optimal:
    print("\nOptimal solution found!")
    
    print(f"\nOptimal Profit: ₹{value(model.objective) * 10000:,.2f}")
    
    print("\nOptimal Distribution Plan:")
    print("From Plant -> To DC: Quantity")
    for i in model.plants:
        for j in model.dcs:
            if value(model.x[i,j]) > 0.01: 
                print(f"Plant {i} -> DC {j}: {value(model.x[i,j]):,.1f}")
    
    print("\nTotal Production per Plant:")
    for i in model.plants:
        total = sum(value(model.x[i,j]) for j in model.dcs)
        print(f"Plant {i}: {total:,.1f}")
        
else:
    print("No optimal solution found.")


Optimal solution found!

Optimal Profit: ₹30,652,000.00

Optimal Distribution Plan:
From Plant -> To DC: Quantity
Plant 1 -> DC 1: 7.0
Plant 1 -> DC 3: 40.0
Plant 1 -> DC 8: 15.0
Plant 1 -> DC 9: 18.0
Plant 1 -> DC 12: 40.0
Plant 2 -> DC 1: 13.0
Plant 2 -> DC 4: 35.0
Plant 2 -> DC 10: 12.0
Plant 2 -> DC 11: 30.0
Plant 3 -> DC 2: 30.0
Plant 3 -> DC 5: 15.0
Plant 3 -> DC 6: 19.0
Plant 3 -> DC 7: 25.0
Plant 3 -> DC 8: 5.0

Total Production per Plant:
Plant 1: 120.0
Plant 2: 90.0
Plant 3: 94.0


In [10]:
results_df = pd.DataFrame(
    [[value(model.x[i,j]) for j in model.dcs] for i in model.plants],
    index=['Plant 1', 'Plant 2', 'Plant 3'],
    columns=[f'DC-{j}' for j in model.dcs]
)
print("\nDetailed Distribution Matrix:")
print(results_df.round(2))



Detailed Distribution Matrix:
         DC-1  DC-2  DC-3  DC-4  DC-5  DC-6  DC-7  DC-8  DC-9  DC-10  DC-11  \
Plant 1   7.0   0.0  40.0   0.0   0.0   0.0   0.0  15.0  18.0    0.0    0.0   
Plant 2  13.0   0.0   0.0  35.0   0.0   0.0   0.0   0.0   0.0   12.0   30.0   
Plant 3   0.0  30.0   0.0   0.0  15.0  19.0  25.0   5.0   0.0    0.0    0.0   

         DC-12  
Plant 1   40.0  
Plant 2    0.0  
Plant 3    0.0  
